In [24]:
from sklearn import tree
from sklearn import metrics
from sklearn import base
from sklearn.ensemble import RandomForestRegressor
import pandas as pd
import os
import numpy as np
import random
import pydotplus
import joblib
from SubjectSysInfo import SubjectSysInfo

file_sep = os.sep
feature_num = None
feature_name = None

sys_optionList = {}


In [25]:
def getFilePath(sub_sys):
	return '.' + file_sep + 'optionfeature' + file_sep + \
            'optioncollection' + file_sep + sub_sys + '.csv'


In [26]:
def processOriginalDataset(dataset):
	column_num = dataset.shape[1]

	new_dataset = dataset.iloc[:, :column_num-1]
	names = ['Io', 'NewArr', "Thread", "Lock"]
	for name in names:
		new_dataset[name] = dataset['directly' + name + 'Sink'] + \
			dataset['if' + name + 'Sink'] + dataset['loop' + name + 'Sink']
	for i in range(0, 4):
		for j in range(i+1, 4):
			namei = names[i]
			namej = names[j]
			new_dataset['sum_' + namei + "_" + namej] = new_dataset[namei] + new_dataset[namej]
	new_dataset['sum_All'] = new_dataset['Io'] + new_dataset['NewArr'] + new_dataset['Thread'] + new_dataset['Lock']

	new_dataset = pd.concat([new_dataset, dataset.iloc[:, column_num-1]], axis=1)

	# print(dataset)
	return new_dataset


In [27]:
def generateDataset(sub_sys_list):
	X_each_sys = {}
	y_each_sys = {}
	for sub_sys in sub_sys_list:
		file_path = getFilePath(sub_sys)
		dataset = pd.read_csv(file_path)
		sys_optionList[sub_sys] = dataset.iloc[:,0].values
		dataset = processOriginalDataset(dataset)
		global feature_num
		global feature_name
		if feature_num is None:
			print('dataset.shape[1] = ', dataset.shape[1])
			feature_num = dataset.shape[1]
			feature_name = dataset.columns[1:feature_num-1]
# 			print(feature_name)
		X_each_sys[sub_sys] = dataset.iloc[:, 1:feature_num - 1].values
		y_each_sys[sub_sys] = dataset.iloc[:, feature_num - 1].values
	X = np.concatenate(
            ([datasubset for datasubset in X_each_sys.values()]))
	y = np.concatenate(
		([datasubset for datasubset in y_each_sys.values()]))
	return X, y

In [28]:
def predit(regressor, X, y):
	target_predict = regressor.predict(X)
	target_predict_sign = np.sign(target_predict)
	print('y = ', y)
	print('p = ', target_predict_sign)
	accuracy_score = metrics.accuracy_score(y, target_predict_sign)
	return accuracy_score

In [29]:
sys_list = ['h2', 'sunflow', 'catena', 'prevayler', 'batik', 'kanzi', 'dconverter', 'cassandra']
train_X, train_y = generateDataset(sys_list[:5])
test_X, test_y = generateDataset(sys_list[5:7])
loaded_regressor = joblib.load("RandomForest_compressed_86.joblib")
accuracy = predit(loaded_regressor, test_X, test_y)
print('accuracy = ', accuracy)
print('sys_list = ', sys_list)

one_list = [sys_list[7]]
test_X, test_y = generateDataset(one_list)
accuracy = predit(loaded_regressor, test_X, test_y)
print('option list = ', sys_optionList[sys_list[7]])
# print('accuracy = ', accuracy)
print('sys = ', one_list)


dataset.shape[1] =  85
y =  [ 1  1  1  1  1  1 -1 -1 -1 -1  1 -1  1  1 -1  1  1  1  1 -1  1 -1  1]
p =  [-1.  1.  1.  1.  1.  1.  1. -1. -1.  1.  1. -1.  1.  1. -1.  1.  1.  1.
  1. -1.  1. -1.  1.]
accuracy =  0.8695652173913043
sys_list =  ['h2', 'sunflow', 'catena', 'prevayler', 'batik', 'kanzi', 'dconverter', 'cassandra']
y =  [-1  1 -1 -1  1  1  1  1  1  1  1  1  1  1  1  1 -1 -1  1  1 -1  1  1 -1
 -1  1  1 -1 -1  1  1  1  1 -1  1  1 -1  1  1 -1 -1  1  1  1 -1  1  1  1
 -1 -1 -1  1  1 -1 -1  1  1 -1  1  1  1  1 -1  1 -1  1  1]
p =  [ 1. -1. -1. -1.  1.  1. -1. -1.  1.  1.  1. -1. -1.  1. -1.  1. -1.  1.
 -1. -1.  1. -1. -1.  1. -1. -1.  1. -1.  1. -1.  1.  1.  1.  1.  1. -1.
 -1.  1.  1. -1.  1.  1.  1. -1. -1. -1. -1. -1. -1. -1.  1.  1.  1.  1.
  1.  1.  1. -1.  1.  1.  1.  1. -1.  1.  1.  1. -1.]
option list =  ['table_count_warn_threshold' 'counter_cache_keys_to_save'
 'internode_application_send_queue_reserve_global_capacity_in_bytes'
 'internode_tcp_connect_timeout_in_ms' 'c